<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
BERTは文章をトークン化したものを入力すると、各トークンに対応するベクトルを返してくれる
このベクトルを集約することで、文章全体の意味を反映したベクトル(=文章ベクトル)を作ることができる

そして、こうしたベクトルの類似度を求めることで文章同士の内容の類似度を比べられる


文章ベクトルのゲット方法 - 2パターン
1. [CLS]に対応するベクトルを使う
2. BERTからの出力ベクトルの平均を取る

BERTの元論文では1が使われていた
しかし、最近の論文では2の方が性能が良いという結果も出ている
今回は2で行くことにする

'''

'\nBERTは文章をトークン化したものを入力すると、各トークンに対応するベクトルを返してくれる\nこのベクトルを集約することで、文章全体の意味を反映したベクトル(=文章ベクトル)を作ることができる\n\nそして、こうしたベクトルの類似度を求めることで文章同士の内容の類似度を比べられる\n\n\n文章ベクトルのゲット方法 - 2パターン\n1. [CLS]に対応するベクトルを使う\n2. BERTからの出力ベクトルの平均を取る\n\nBERTの元論文では1が使われていた\nしかし、最近の論文では2の方が性能が良いという結果も出ている\n今回は2で行くことにする\n\n'

In [2]:
!mkdir chap10
%cd ./chap10

/content/chap10


In [3]:
# ライブラリインストール
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

import random
import glob
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel

# BERTの日本語モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 35.0 MB/s 
     |████████████████████████████████| 486 kB 69.2 MB/s 
     |████████████████████████████████| 13.4 MB 52.7 MB/s 
     |████████████████████████████████| 120 kB 57.4 MB/s 
     |████████████████████████████████| 880 kB 33.6 MB/s 
     |████████████████████████████████| 6.6 MB 29.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=af783465f9d4ede5591b63051160287f786c221e548059efafb0f8194a0e7a06
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=354e47c02c4df81f9b67704102f609ef4e4b16b13079b2d76b3e0b98afb75dae
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic

In [4]:
# とりあえずデータセットをダウンロードしよう
# livedoorニュースコーパスを使うぞ

#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

--2022-09-21 12:10:19--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.44MB/s    in 1.6s    

2022-09-21 12:10:21 (5.44 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [5]:
'''
データを取り込んで、文章ベクトルを出力しよう

・　コーパス内のカテゴリーのリスト定義
    ここに書いてある9個のカテゴリにニュースが分類されている
    https://www.rondhuit.com/download.html#ldcc
・　トークナイザ・モデルのロード

・　文章を取り出してはトークン化・BERTに入れて、出力の平均を取る

livedoorコーパスのデータ形式

フォルダの位置
./text/カテゴリ名/カテゴリ名-記事ID.txt　

例えば
./text/it-life-hack/it-life-hack-6342280.txt　

txtファイルの中身

# 一行目：URL
# 二行目：作成日時
# 三行目：記事タイトル
# 四行目以降：本文
という構成になっているはず



'''

'\nデータを取り込んで、文章ベクトルを出力しよう\n\n・\u3000コーパス内のカテゴリーのリスト定義\n    ここに書いてある9個のカテゴリにニュースが分類されている\n    https://www.rondhuit.com/download.html#ldcc\n・\u3000トークナイザ・モデルのロード\n\n・\u3000文章を取り出してはトークン化・BERTに入れて、出力の平均を取る\n\nlivedoorコーパスのデータ形式\n\nフォルダの位置\n./text/カテゴリ名/カテゴリ名-記事ID.txt\u3000\n\n例えば\n./text/it-life-hack/it-life-hack-6342280.txt\u3000\n\ntxtファイルの中身\n\n# 一行目：URL\n# 二行目：作成日時\n# 三行目：記事タイトル\n# 四行目以降：本文\nという構成になっているはず\n\n\n\n'

In [8]:
# カテゴリリスト定義
# コーパス中のデータのカテゴリ名一覧
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザとモデルをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)



# 文章を1文ずつ取り出す
for label, category in enumerate(category_list):
    # まずはどのファイルの文章を読み込むか選ぶ
    for file in glob.glob(f'./text/{category}/*'):
        

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
